# Preprocessing

In [1]:
import os
import bestreads.text as text
import pandas as pd

data = pd.read_csv('./data/goodreads_books.csv')
data_test = data.sample(frac = 0.2, random_state = 111)
data_train = data.drop(data_test.index)

data_val = data_train.sample(frac=0.2, random_state=112)
data_train = data_train.drop(data_val.index)

## Train-Test Set Splits

In [2]:
save_dir = './data/processed/'
os.makedirs(save_dir, exist_ok=True)
data_train.to_csv(save_dir + 'goodreads_books_train.csv', index = False)
data_val.to_csv(save_dir + 'goodreads_books_val.csv', index=False)
data_test.to_csv(save_dir + 'goodreads_books_test.csv', index = False)
data_train.reset_index(inplace=True)
data_val.reset_index(inplace=True)
data_test.reset_index(inplace=True)

## Text Cleaning

In [3]:
is_english_description_train = text.is_english(data_train['description'])
english_descriptions_train = data_train.description[is_english_description_train]
cleaned_descriptions_train = text.clean_text(english_descriptions_train)
is_english_description_train = is_english_description_train.rename('is_english_description',)
cleaned_descriptions_train = cleaned_descriptions_train.rename('cleaned_descriptions')

is_english_description_val = text.is_english(data_val['description'])
english_descriptions_val = data_val.description[is_english_description_val]
cleaned_descriptions_val = text.clean_text(english_descriptions_val)
is_english_description_val = is_english_description_val.rename('is_english_description',)
cleaned_descriptions_val = cleaned_descriptions_val.rename('cleaned_descriptions')

is_english_description_test = text.is_english(data_test['description'])
english_descriptions_test = data_test.description[is_english_description_test]
cleaned_descriptions_test = text.clean_text(english_descriptions_test)
is_english_description_test = is_english_description_test.rename('is_english_description')
cleaned_descriptions_test = cleaned_descriptions_test.rename('cleaned_descriptions')

## Grouping Descriptions by Genre

In [4]:
genre_and_votes_train = text.get_genres(data_train.genre_and_votes[is_english_description_train])
genre_and_votes_val = text.get_genres(data_val.genre_and_votes[is_english_description_val])
genre_and_votes_test = text.get_genres(data_test.genre_and_votes[is_english_description_test])
combined = text.combine_genres(genre_and_votes_train.genre_1, cleaned_descriptions_train)

/home/adam/Documents/bestreads/bestreads/text.py:144: RuntimeWarning: NaN values detected in genre_and_votes; these will beskipped
  + 'skipped', category=RuntimeWarning)
100%|████████████████████████████████████| 8777/8777 [00:00<00:00, 18143.92it/s]


## Calculating TF-IDF

In [5]:
tf_idf_table_train = text.tf_idf(combined)

# Here, we save the index because each index is a word
tf_idf_table_train.to_csv(save_dir + 'tf_idf_table_train.csv', index_label='word')
tf_idf_table_train.head()

100%|███████████████████████████████████████████| 52/52 [00:11<00:00,  4.53it/s]


,Speculative Fiction,Historical,Writing,Reference,War,Shapeshifters,Race,Plays,Horror,Holiday,...,Politics,Poetry,LGBT,Literature,Paranormal,Science Fiction,Romance,Fantasy,Space,Literary Fiction
thi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
futur,0.000277,0.000045,0.000029,0.000005,0.000038,0.000052,0.000017,0.000043,0.000014,0.000000,...,0.000054,0.000010,0.000028,0.000014,0.000006,0.000125,0.000063,0.000065,0.000108,0.000022
peopl,0.000064,0.000040,0.000030,0.000020,0.000029,0.000018,0.000059,0.000022,0.000046,0.000000,...,0.000040,0.000008,0.000023,0.000027,0.000012,0.000034,0.000022,0.000026,0.000028,0.000027
need,0.000059,0.000037,0.000044,0.000131,0.000075,0.000195,0.000012,0.000026,0.000085,0.000000,...,0.000073,0.000023,0.000131,0.000017,0.000079,0.000080,0.000156,0.000097,0.000047,0.000040
sleep,0.000193,0.000035,0.000036,0.000123,0.000050,0.000053,0.000000,0.000021,0.000124,0.000222,...,0.000000,0.000017,0.000000,0.000024,0.000054,0.000023,0.000040,0.000000,0.000085,0.000000


In [6]:
# Save the training data
processed_data_train = (is_english_description_train.to_frame()
                        .merge(cleaned_descriptions_train,
                               left_index=True, right_index=True,
                               validate='1:1')
                        .merge(genre_and_votes_train,
                               left_index=True, right_index=True,
                               validate='1:1'))

# Here, we save the index because some rows are now missing due to
# english language selection
processed_data_train.to_csv(save_dir + 'goodreads_books_train_processed.csv', 
                            index_label='index')

# Save the validation
processed_data_val = (is_english_description_val.to_frame()
                        .merge(cleaned_descriptions_val,
                               left_index=True, right_index=True,
                               validate='1:1')
                        .merge(genre_and_votes_val,
                               left_index=True, right_index=True,
                               validate='1:1'))

# Here, we save the index because some rows are now missing due to
# english language selection
processed_data_val.to_csv(save_dir + 'goodreads_books_val_processed.csv', 
                          index_label='index')

# Save the test data
processed_data_test = (is_english_description_test.to_frame()
                        .merge(cleaned_descriptions_test,
                               left_index=True, right_index=True,
                               validate='1:1')
                        .merge(genre_and_votes_test,
                               left_index=True, right_index=True,
                               validate='1:1'))

# Here, we save the index because some rows are now missing due to
# english language selection
processed_data_test.to_csv(save_dir + 'goodreads_books_test_processed.csv', 
                           index_label='index')